In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.manual_seed(1)

In [5]:
batch_size = 128
num_classes = 8

In [ ]:
import numpy as np
import tensorflow as tf
import copy
import pandas as pd

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_val = np.moveaxis(x_train, [3, 1, 2], [1, 2, 3]).astype('float32')[45000:50000]
x_train = np.moveaxis(x_train, [3, 1, 2], [1, 2, 3]).astype('float32')[:45000]
x_test = np.moveaxis(x_test, [3, 1, 2], [1, 2, 3]).astype('float32')
 
x_train /= 255
x_val /= 255
x_test /= 255

    
#８〜９ラベルを１〜７に
y_val = y_train.reshape(-1).astype('long')[45000:50000]
y_val_rand = copy.copy(y_val)
y_val_rand[y_val_rand > 7] = np.random.randint(0,8,len(y_val_rand[y_val_rand > 7]))

y_train = y_train.reshape(-1).astype('long')[:45000]
y_train_rand = copy.copy(y_train)
noisesample = np.where(y_train_rand > 7)[0]
cleansample = np.where(y_train_rand <= 7)[0]
y_train_rand[y_train_rand > 7] = np.random.randint(0,8,len(noisesample))
                                                   
dfnoise = pd.DataFrame([noisesample])
dfclean = pd.DataFrame([cleansample])                                                                     
    
y_test = y_test.reshape(-1).astype('long')

testsample = np.where(y_test <= 7)[0]
y_test = y_test[testsample]
x_test = x_test[testsample]

ds_like = data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train_rand))
dataloader_like = data.DataLoader(dataset=ds_like, batch_size=1000, shuffle=False)

ds_val = data.TensorDataset(torch.from_numpy(x_val), torch.from_numpy(y_val_rand))
dataloader_val = data.DataLoader(dataset=ds_val, batch_size=128, shuffle=False)

ds_test = data.TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))
dataloader_test = data.DataLoader(dataset=ds_test, batch_size=128, shuffle=True)

In [7]:
model = models.mobilenet_v2(progress=True, **{"num_classes": 8}).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

In [11]:
global_step = 0
 
def train(epoch, writer):
    model.train()
    scheduler.step()    
    
    steps = len(ds_sel)//batch_size
    for step, (images, labels) in enumerate(dataloader_sel, 1):
        global global_step
        global_step += 1
 
        images, labels = images.to(device), labels.to(device)
 
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        if step % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch, epochs, step, steps, loss.item()))
            writer.add_scalar('train/train_loss', loss.item() , global_step)

In [12]:
import numpy as np
def softmax(Llist):

    exp_x = np.exp(Llist)    
    y = exp_x / np.array([np.sum(exp_x,axis=1)]).T    
    return np.max(y,axis=1)

In [14]:
def expert(epoch):
    model.eval()
    correct = 0
    total = 0
    labelcheck = []
    with torch.no_grad():
        p_list = []
        for step,(images, labels) in enumerate(dataloader_like,1):
            labelinf = np.zeros(len(labels))
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
           
            _, predicted = torch.max(outputs.data, 1)
           
            p = softmax([list(map(float,i)) for i in list(outputs)])
            p_list.extend(p)
            
            prelist = list(map(int,predicted))
            lablist = list(map(int,labels))
            
            labelinf[np.where(np.array(prelist) == np.array(lablist))[0]] = 1
            labelinf[np.where(np.array(prelist) != np.array(lablist))[0]] = -1
            
            labelcheck.extend(labelinf)
            

            if step % 100 == 0:
                print("PP:"+str(p[0]))
            
        train_acc = len(np.where(np.array(labelcheck)==1)[0])/len(labelcheck)
        loss_list = (1 - np.array(labelcheck)*(1 - np.array(p_list)))/2
        print("Tra Acc : %.4f" % (train_acc))
    
    return np.array(loss_list),train_acc

In [15]:
def eval(epoch,writer):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for (images, labels) in dataloader_val:
            images, labels = images.to(device), labels.to(device)
 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
 
    print("Val Acc : %.4f" % (correct/total))
    writer.add_scalar('eval/val_acc', correct*100/total, epoch)
    
    return correct/total

In [16]:
def test(epoch,writer):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for (images, labels) in dataloader_test:
            images, labels = images.to(device), labels.to(device)
 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            #print(predicted)
            #print(outputs)
 
    print("Test Acc : %.4f" % (correct/total))
    writer.add_scalar('eval/val_acc', correct*100/total, epoch)
    
    return correct/total

In [ ]:
from tensorboardX import SummaryWriter
from statistics import mean
import random
import pandas as pd
writer = SummaryWriter()

batch_size = 128
epochs = 200

n = len(y_train_rand)


M = 4*np.log(n)
β = 1/(1+np.sqrt(2*np.log(n)/M))

for k in [22500,27000,31500,36000,40500,45000]:
    
    eta = np.sqrt(k*epochs)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    percent = (100*k)/45000

    train_acclist = []
    val_acclist = []
    test_acclist = []
    cumloss_list = []

    xt = np.array(random.sample(range(0,n),k=k))
    
    if k < 45000:
        
        cumloss = np.zeros(n)
        c_map = np.array([1/n for j in range(n)])
        
        for epoch in range(1, epochs+1):
    
            xt = np.sort(xt)
            c = np.zeros(n)
            c[xt] = 1
            c_map = np.vstack((c_map,c))
    
            ds_sel = data.TensorDataset(torch.from_numpy(x_train[xt]), torch.from_numpy(y_train_rand[xt]))
            dataloader_sel = data.DataLoader(dataset=ds_sel, batch_size=batch_size, shuffle=True)
    
            train(epoch, writer)
            loss_list,train_acc = expert(epoch)
            cumloss = cumloss + loss_list
            cumloss_list.append(cumloss)
            normalnoise = np.random.normal(0,0.00001,(n))
            virtualloss = cumloss + eta*normalnoise 
            xt = np.argsort(virtualloss)[:k]
            
            
            val_acc = eval(epoch, writer)
            test_acc = test(epoch, writer)
    
           
            train_acclist.append(train_acc)
            val_acclist.append(val_acc)
            test_acclist.append(test_acc)
    
    elif k == 45000:

        ds_sel = data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train_rand))
        dataloader_sel= data.DataLoader(dataset=ds_sel, batch_size=batch_size, shuffle=True)

        for epoch in range(1, epochs+1):
            train(epoch, writer)
            likelihoodlist,train_acc = expert(epoch)
            val_acc = eval(epoch, writer)
            test_acc = test(epoch, writer)
    
            train_acclist.append(train_acc)
            val_acclist.append(val_acc)
            test_acclist.append(test_acc)
    
writer.close()
print("END")